In [1]:
import torch
import os, sys
from tqdm import tqdm
module_rootdir = '.'
dataset_rootdir = '../'
label_rootdir = module_rootdir
weight_rootdir = module_rootdir
sys.path.append(module_rootdir)

DET_DIR = 'mAP/input/detection-results'
GT_DIR = 'mAP/input/ground-truth'

from modules.proposal.box_association import  identify_prominent_objects
from modules.first_stage.inference import inference
from modules.first_stage.set_parameters_for_inference import set_param_for_inference
from modules.plot.viz_annotation import vizualize_bbox_resized
from modules.dataset_utils.kitti_dataset_utils.constants import _IDX_TO_OBJ_CLASS_

In [2]:
weights_file = 'model_weights/1705990924432/anchor_free_detector.pt'

param_dict = set_param_for_inference(
    dataset_type = 'kitti',
    module_rootdir = module_rootdir,
    dataset_rootdir = dataset_rootdir,
    label_rootdir = label_rootdir,
    batch_size = 1,
    trained_weights_file = os.path.join(weight_rootdir, weights_file))

device = param_dict['device']
dataset_param = param_dict['dataset_param']
dataset_train = param_dict['dataset_train']
dataset_val = param_dict['dataset_val'] 
detector = param_dict['detector']

deltas_mean = torch.tensor(dataset_param.deltas_mean, dtype=torch.float32, device=device)
deltas_std = torch.tensor(dataset_param.deltas_std, dtype=torch.float32, device=device)
grid_coord = dataset_param.grid_coord.to(device)
ignored_classId = dataset_param.ignored_classId

GPU is available. Good to go!
printing model config parameters
----------------------------------------------------------------------------------------------------
backbone                        : efficientnet_b4
num_backbone_nodes              : 4
num_extra_blocks                : 1
num_levels                      : 5
extra_blocks_feat_dim           : 512
num_fpn_blocks                  : 2
fpn_feat_dim                    : 128
prediction head stem_channels   : [128, 128, 128, 128]
activation                      : swish
image dimension BDD (H, W, D)   : (360, 640, 3)
image dimension KITTI (H, W, D) : (263, 873, 3)
num_classes                     : 2
DEVICE                          : cuda
****************************************************************************************************
 
Loading JSON file .. please wait
Sequence: 0000
Sequence: 0001
Sequence: 0002
Sequence: 0003
Sequence: 0004
Sequence: 0005
Sequence: 0006
Sequence: 0008
Sequence: 0009
Sequence: 0011
Sequence: 0012

In [3]:
det_dir = os.path.join(module_rootdir, DET_DIR)
gt_dir = os.path.join(module_rootdir, GT_DIR)

if not os.path.exists(det_dir): 
    os.makedirs(det_dir, exist_ok=True)

if not os.path.exists(gt_dir): 
    os.makedirs(gt_dir, exist_ok=True)

# dataset = dataset_train
dataset = dataset_val
iter_start_offset = 0
max_iters = iter_start_offset + len(dataset)

nms_thresh = 0.35
# score_threshold = torch.tensor([0.5, 0.4], dtype=torch.float32).to(device)
# score_threshold = torch.tensor([0.6, 0.5], dtype=torch.float32).to(device)
# score_threshold = torch.tensor([0.7, 0.6], dtype=torch.float32).to(device)
score_threshold = torch.tensor([0.8, 0.7], dtype=torch.float32).to(device)

for iter in tqdm(range(iter_start_offset, max_iters)):

    img, labels = dataset.__getitem__(iter)
    img_path = labels['img_path']
    img = img.unsqueeze(0).to(device)
    bboxes = labels['bbox'].to(device)
    clslabels = labels['obj_class_label'].to(device)

    valid_gt_objects_flag = identify_prominent_objects(bboxes) & (clslabels != ignored_classId) 
    bboxes = bboxes[valid_gt_objects_flag]
    clslabels = clslabels[valid_gt_objects_flag]

    pred = inference(
        detector, img, grid_coord, 
        deltas_mean, deltas_std,
        score_threshold, nms_thresh)
    
    pred_score = pred['pred_score'].cpu().numpy()
    pred_class = pred['pred_class'].cpu().numpy()
    pred_box = pred['pred_box'].cpu().numpy()
    gt_class = clslabels.cpu().numpy()
    gt_box = bboxes.cpu().numpy()
    gt_class = [_IDX_TO_OBJ_CLASS_[int(i)] for i in gt_class]
    pred_class = [_IDX_TO_OBJ_CLASS_[int(i)] for i in pred_class]

    # Skip current iteration if no predictions were found.
    if pred_box.shape[0] == 0:
        continue

    file_name = str(iter) + '.txt'

    with open(os.path.join(det_dir, file_name), "w") as file_det: 
          for label, box, conf in zip(pred_class, pred_box, pred_score):
                line = f"{label} {conf:.6f} {box[0]:.2f} {box[1]:.2f} {box[2]:.2f} {box[3]:.2f}\n"
                file_det.write(line)

    with open(os.path.join(gt_dir, file_name), "w") as file_gt: 
          for label, box in zip(gt_class, gt_box):
                line = f"{label} {box[0]:.2f} {box[1]:.2f} {box[2]:.2f} {box[3]:.2f}\n"
                file_gt.write(line)

100%|██████████| 1879/1879 [02:11<00:00, 14.26it/s]


In [4]:
%run ./mAP/main.py

59.50% = person AP 
79.50% = vehicle AP 
mAP = 69.50%
